In [1]:
import os, sys
PWD = os.getenv ('PWD')


def init_django(project_name=None) :
    os.chdir (PWD)
    project_name = project_name or os.environ.get (project_name)

    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE" ] = "true"
    import django
    django. setup ()
init_django("OnlineShop")

In [2]:
import pandas as pd
import numpy as np

In [5]:
from ProductFeed.models import *

In [6]:
all_clothes = Clothes.objects.all()

In [199]:
# поля таблицы Clothes
all_fields = [field.name for field in Clothes._meta.get_fields(include_hidden=True)]
all_fields.remove("title")
all_fields.remove("photo")
all_fields.remove("size")
all_fields.remove("season")
all_fields.remove("style")
all_fields.remove("subcategory")
all_fields.remove("brand")
all_fields.remove("country")
all_fields.remove("Clothes_size+")

In [218]:
all_seasons = [s.__str__() for s in Seasons.objects.all()]
all_styles = [s.__str__() for s in Styles.objects.all()]

In [235]:
# формируем pd.DataFrame()
df = pd.DataFrame(columns=all_fields)
df

,Clothes_season+,Clothes_style+,Clothes_subcategory+,Clothes_country+,Clothes_brand+,id,cost,category,color,by_count,is_premium,gender,count_in_stock


In [236]:
for cl in all_clothes:
    obj = {}
    for f in all_fields:
        if f[-1] != "+":
            v = getattr(cl, f)
            obj[f] = v
        else:
            new_f = f[f.index("_")+1:]
            new_f = new_f[:-1]
            
            all_values = getattr(cl, new_f).all()
            value = []
            for elem in all_values:
                value.append(elem.__str__())
            if new_f not in ["season", "style"]:
                obj[f] = str(value[0])
            else:
                if new_f == "season":
                    for s in all_seasons:
                        obj["season" + s] = int(s in value)
                else:
                    for st in all_styles:
                        obj["style" + st] = int(st in value)
    new_df = pd.DataFrame([obj])
    df = pd.concat([df, new_df], ignore_index=True)
df = df.drop(["Clothes_season+", "Clothes_style+"], axis=1)
df = df.set_index("id")
df["cost"] = df["cost"].astype(float)
df["count_in_stock"] = df["count_in_stock"].astype(int)

In [237]:
df

,Clothes_subcategory+,Clothes_country+,Clothes_brand+,cost,category,color,by_count,is_premium,gender,count_in_stock,seasonЗима,seasonОсень,seasonЛето,seasonВесна,seasonДемисезон,styleСпортивный,styleПовседневный,styleДеловой,styleКлассический
id,,,,,,,,,,,,,,,,,,,
1,Худи,США,Topman,3000.0,Одежда,blue,10,False,Male,10,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,Рубашка,Германия,Mia Mania,2200.0,Одежда,white,1,False,Male,10,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Футболка,Россия,marhatter,1200.0,Одежда,color,0,False,Male,10,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,Кроссовки,США,Reebok,5500.0,Обувь,black,1,False,Male,10,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
5,Кардиган,Германия,Mademan,2250.0,Одежда,brown,0,False,Male,10,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
6,Джемпер,Германия,Tom Tailor,7990.0,Одежда,gray,0,False,Male,10,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
7,Брюки,Германия,Mademan,4200.0,Одежда,black,0,False,Male,10,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
8,Штаны,Германия,Carhatt,5500.0,Одежда,pink,1,False,Male,10,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
9,Пиджак,Германия,Qwentiny,5749.0,Одежда,black,0,False,Male,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0


In [238]:
from sklearn.preprocessing import LabelEncoder
import joblib as jb

In [239]:
for col in df.columns:
    if df[col].dtype == "object":
        encoder = LabelEncoder()
        encoder.fit(df[col])
        jb.dump(encoder, f"{col}_encoder.joblib")
        df[col] = encoder.transform(df[col])
        

In [240]:
df

,Clothes_subcategory+,Clothes_country+,Clothes_brand+,cost,category,color,by_count,is_premium,gender,count_in_stock,seasonЗима,seasonОсень,seasonЛето,seasonВесна,seasonДемисезон,styleСпортивный,styleПовседневный,styleДеловой,styleКлассический
id,,,,,,,,,,,,,,,,,,,
1,20,3,18,3000.0,2,2,5,0,1,10,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,14,1,7,2200.0,2,10,1,0,1,10,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,19,2,19,1200.0,2,4,0,0,1,10,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,9,3,11,5500.0,1,1,1,0,1,10,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
5,7,1,6,2250.0,2,3,0,0,1,10,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
6,4,1,16,7990.0,2,5,0,0,1,10,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
7,2,1,6,4200.0,2,1,0,0,1,10,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
8,22,1,1,5500.0,2,7,1,0,1,10,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
9,11,1,10,5749.0,2,1,0,0,1,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0


In [285]:
class MyRecSystem():
    def __init__(self):
        self.data = None

    def fit(self, X):
        self.data = X

    def predict(self, index_obj):
        assert(self.data is not None), "Сначала нужно вызвать метод fit"
        
            
        all_obj = []
        for i in range(len(self.data)):
            if i != index_obj:
                cur_dist = self.cos_dist(self.data[index_obj], self.data[i])
                all_obj.append([cur_dist, i])
        all_obj.sort()
        return all_obj
    
    def cos_dist(self, x, y) -> float:
        cosine_similarity = np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))
        return cosine_similarity

In [286]:
model = MyRecSystem()

In [287]:
model.fit(df.to_numpy())

In [288]:
model.predict(1)

[[np.float64(0.9998659597753243), 2],
 [np.float64(0.9998757544531678), 16],
 [np.float64(0.9999086164530754), 25],
 [np.float64(0.9999255212647525), 13],
 [np.float64(0.9999463052063499), 23],
 [np.float64(0.9999542618260153), 33],
 [np.float64(0.9999577686109097), 31],
 [np.float64(0.9999582204018839), 14],
 [np.float64(0.9999608376622362), 12],
 [np.float64(0.9999650729524493), 20],
 [np.float64(0.999968019177068), 30],
 [np.float64(0.9999684187437976), 5],
 [np.float64(0.999969059715182), 6],
 [np.float64(0.9999691100556034), 32],
 [np.float64(0.9999700092922027), 28],
 [np.float64(0.9999704607531884), 21],
 [np.float64(0.9999712269132888), 29],
 [np.float64(0.9999713536965613), 24],
 [np.float64(0.9999738452880925), 18],
 [np.float64(0.9999743906327957), 3],
 [np.float64(0.9999748345805443), 26],
 [np.float64(0.9999749760828394), 8],
 [np.float64(0.9999751885752366), 17],
 [np.float64(0.9999760266419415), 19],
 [np.float64(0.9999772587947867), 11],
 [np.float64(0.9999832164993763)

In [289]:
jb.dump(model, "Rec_system.joblib")

['Rec_system.joblib']